# Assignment 3, Problem 3

In [189]:
#Import packages
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import math
import os
from openpyxl import load_workbook

In [190]:
#set directories
os.chdir("/Users/briangoggin/Dropbox/CP 201A/Fruitvale")
cwd = os.getcwd()
root = cwd #root is directory to raw files


In [191]:
#import data
poverty = root+"/Raw Data/ACS_14_5YR_B17001_with_ann.csv"
acs = pd.read_csv(poverty, header = 1)



In [192]:
#identify Monument Census Tracts
Fruitvale = [6001406100, 6001406201, 6001406202, 6001406300, 6001406500, 6001407101, 6001407102, 6001407200, 6001407300, 6001407400, 6001407500, 6001407600]

acs['Fruitvale'] = acs['Id2'].isin(Fruitvale)

In [193]:
#Define function for proportions MOE
def prop_MOE(numerator, denominator, num_moe, den_moe):
    return ((num_moe**2 - (((numerator/denominator)**2)*(den_moe**2))))**(1/2)/denominator

In [194]:
#############################
##Section 1: ACS Data
#############################

acs['total'] = acs["Estimate; Total:"].astype(int)
acs['total_moe'] = acs["Margin of Error; Total:"].astype(int)
acs['poor'] = acs["Estimate; Income in the past 12 months below poverty level:"].astype(int)
acs['poor_moe'] = acs["Margin of Error; Income in the past 12 months below poverty level:"].astype(int)

#prepare variables for collapse
acs['total_moe2'] = acs['total_moe']**2
acs['poor_moe2'] = acs['poor_moe']**2

In [195]:
#sumdata
sumvars = ['total', 'total_moe2', 'poor', 'poor_moe2']
acs_sum = acs.groupby('Fruitvale')[sumvars].sum()
acs_sum['total_moe'] = acs_sum['total_moe2']**(1/2)
acs_sum['poor_moe'] = acs_sum['poor_moe2']**(1/2)
acs_sum['pct_poor'] = (acs_sum['poor']/acs_sum['total'])
acs_sum['pct_poor_moe'] = prop_MOE(acs_sum['poor'], acs_sum['total'],  acs_sum['poor_moe'], acs_sum['total_moe'])

In [196]:
#final cleaning
acs_sum = acs_sum.round(4)
acs_sum['pct_poor'] = 100*acs_sum['pct_poor']
acs_sum['pct_poor_moe'] = 100*acs_sum['pct_poor_moe']
acs_sum['survey'] = "ACS"

In [197]:
acs_sum.head()

,total,total_moe2,poor,poor_moe2,total_moe,poor_moe,pct_poor,pct_poor_moe,survey
Fruitvale,,,,,,,,,
False,1476841,48400946,181616,24967626,6957.0788,4996.7616,12.30,0.33,ACS
True,54505,4121028,15575,2249193,2030.0315,1499.7310,28.58,2.54,ACS


In [198]:
#############################
##Section 2: SF3 Data
#############################

#redefine Fruitvale for Fruitvale Census Tracts in 2000
Fruitvale = [6001406100, 6001406201, 6001406202, 6001406300, 6001406500, 6001407100, 6001407200, 6001407300, 6001407400, 6001407500, 6001407600]

#import data
poverty = root+"/Raw Data/DEC_00_SF3_PCT049_with_ann.csv"
sf3 = pd.read_csv(poverty, header = 1)
sf3['Fruitvale'] = sf3['Id2'].isin(Fruitvale)


In [199]:
#rename vars
sf3['total'] = sf3["Total:"].astype(int)
sf3['poor'] = sf3["Income in 1999 below poverty level:"].astype(int)


In [200]:
#collapse data
sumvars = ['total', 'poor']
sf3_sum = sf3.groupby('Fruitvale')[sumvars].sum()
sf3_sum['total_moe'] = acs_sum['total_moe']
sf3_sum['poor_moe'] = acs_sum['poor_moe']
sf3_sum['pct_poor'] = (sf3_sum['poor']/sf3_sum['total'])
sf3_sum['pct_poor_moe'] = prop_MOE(sf3_sum['poor'], sf3_sum['total'],  sf3_sum['poor_moe'], sf3_sum['total_moe'])


In [201]:
#final cleaning
sf3_sum = sf3_sum.round(4)
sf3_sum['pct_poor'] = 100*sf3_sum['pct_poor']
sf3_sum['pct_poor_moe'] = 100*sf3_sum['pct_poor_moe']
sf3_sum['survey'] = "SF3"

In [202]:
#############################
##Section 3: Append data
#############################

final = acs_sum.append(sf3_sum)
final = final.drop(final.index == "False") #keep only Fruitvale 
keepvars = ['pct_poor', 'pct_poor_moe', 'survey'] #only keep 2 columns for final output
final = final[keepvars]

In [203]:
#export data to csv
export_path = cwd+'/Output/python_output/poverty_over_time.csv'
final.to_csv(export_path)